In [123]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import pandas as pd
import sys
import numpy as np


k = 2


def read_file(filename):
    return pd.read_csv(filename, index_col=0, parse_dates=True, infer_datetime_format=True)


def get_data(timeframe, name):
    return read_file("data_with_new_indicators/btcusdt_" + timeframe + "_" + name + ".csv")


# Tick Size of Data
time_frame = "1h"
# Load data
df_train = get_data(time_frame, "train")
df_val = get_data(time_frame, "val")


# Create the linear regression model
model = LinearRegression()


# Define the independent variables
X_train = df_train.drop(f'Next_{k}_Days_Return', axis=1)
X_train = X_train.drop('open', axis=1)
X_train = X_train.drop('close', axis=1)
X_val = df_val.drop(f'Next_{k}_Days_Return', axis=1)
X_val = X_val.drop('open', axis=1)
X_val = X_val.drop('close', axis=1)


# Define the dependent variable
y_train = df_train[f'Next_{k}_Days_Return']
y_val = df_val[f'Next_{k}_Days_Return']
# print(X_train.shape, y_train.shape)
# print(X_val.shape, y_val.shape)


df_train['flag'] = 0
df_val['flag'] = 0
# Fit the linear regression model on the training data


model.fit(X_train, y_train)
print("model_coef", model.coef_)


# Predict the dependent variable of the training data
y_pred = model.predict(X_train)
quantile_value = np.quantile(np.abs(y_pred), 0.93)
print("quantile value: ", quantile_value)
print(np.corrcoef(y_pred, y_train)[0,1])
quantile_value = 1.6
# print(df_train['datetime'].head())
df_train['date_time'] = list(pd.to_datetime(pd.Series(df_train.index)))
df_val['date_time'] = list(pd.to_datetime(pd.Series(df_val.index)))

# df_train['flag'] = np.where(y_pred > quantile_value, 1, np.where(y_pred < -quantile_value, -1, 0))
# data = df_train
# tot = 0
# for index, _ in data.iterrows():
#     data.at[index, 'signal'] = 0
#     if data.at[index, 'flag'] == 1:
#         if tot == 0 or tot == -1:
#             # enter long position/exit short position
#             tot += 1
#             data.at[index, 'signal'] = 1
#     elif data.at[index, 'flag'] == -1:
#         if tot == 0 or tot == 1:
#             # enter short position/exit long position
#             tot -= 1
#             data.at[index, 'signal'] = -1

# data[data['signal']!=0][:40]



/tmp/ipykernel_3770/598726052.py:11: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(filename, index_col=0, parse_dates=True, infer_datetime_format=True)
/tmp/ipykernel_3770/598726052.py:11: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  return pd.read_csv(filename, index_col=0, parse_dates=True, infer_datetime_format=True)


model_coef [8.69017981e-05 2.13112354e-02 2.15307275e+00 1.02680307e+00
 1.15511929e+00]
quantile value:  1.634936680497444
0.10185396648465336


In [124]:

df_train['flag'] = np.where(y_pred > quantile_value, 1, np.where(y_pred < -quantile_value, -1, 0))
# old_df_train = df_train
df_fine = pd.read_csv("data_with_new_indicators/btcusdt_3m_train.csv")
df_train = df_train.drop(columns= ["open", "close", "volume"])
df_fine["datetime"] = pd.to_datetime(df_fine["datetime"])
df_fine.rename(columns={'datetime': 'date_time'}, inplace=True)

df_fine['date_time'] = df_fine['date_time'] - pd.DateOffset(minutes=55)


df_train = df_fine.merge(df_train[['date_time', 'flag']], on='date_time', how='left')
df_train.fillna(0, inplace=True)

# df_train.loc[df_train['date_time'] == old, 'close'] = list(old_df_train.loc[old_df_train['flag'] != 0, 'close'])[:-1]
df_train[df_train['flag'] != 0][:40]



,date_time,open,close,volume,Daily_Return,Next_2_Days_Return,RSI,exp_RSI,EMA_Slope,flag
59,2018-01-01 09:30:00,13575.76,13601.01,55.300506,1.694653,0.477170,0.0,0.002934,-1.318575,-1.0
83,2018-01-01 11:30:00,13774.09,13780.41,21.226193,0.473359,1.784417,0.0,0.002934,-0.667269,-1.0
143,2018-01-01 16:30:00,13199.95,13220.56,27.017572,1.632698,5.660880,0.0,0.002934,-1.356562,1.0
167,2018-01-01 18:30:00,12968.53,13017.00,25.255931,3.739057,-13.598371,0.0,0.002934,-0.989330,1.0
251,2018-01-02 01:30:00,13422.78,13399.24,21.525367,-1.703905,3.964404,0.0,0.002934,0.716906,-1.0
299,2018-01-02 05:30:00,13750.01,13750.01,25.736793,0.000000,0.725090,0.0,0.002934,-0.131681,-1.0
347,2018-01-02 09:30:00,13163.85,13127.31,52.891994,-4.742279,6.455245,0.0,0.002934,0.784236,1.0
455,2018-01-02 18:30:00,13847.49,13864.98,82.571610,1.263045,-4.038232,0.0,0.002934,-0.873736,-1.0
515,2018-01-02 23:30:00,14584.03,14580.00,71.910920,-0.675127,3.637860,0.0,0.002934,0.588106,-1.0
527,2018-01-03 00:30:00,14660.53,14732.75,95.528923,4.924781,11.354296,0.0,0.002934,0.279377,-1.0


In [125]:

# print(df_train.head())
exit_loss = 0
disable_trading = 0
# stop_loss = 0
compare = 0
thresh = -float(sys.argv[2])
logs = []
for i in range(len(df_train)):
    if df_train["flag"].iloc[i] == 1 and compare == 0:
        # No open trade, encouter buy singal
        exit_loss = 0
        buy_price = df_train["close"].iloc[i]
        
        compare = 1
        logs.append(1)

    #Once we close a trade, we have to check in df_fine whether to exit or not

    elif (df_train["flag"].iloc[i] != -1 and compare == 1):
        # Current buy trade, encounter buy signal or no signal - do nothing, update stop loss
        logs.append(0)

        #calculate pnl, if we exit now
        sell_price = df_train["close"].iloc[i]
        exit_loss = (sell_price - buy_price)/buy_price 
        

        #exit trade, if the loss is higher than stop loss
        if exit_loss < thresh and disable_trading == 0:
            logs[-1] = -1
            # print(f"disable_trading in buy trade - stop loss: {disable_trading}")
            disable_trading = 1


    elif df_train["flag"].iloc[i] == -1 and compare == 1:
        # Current buy trade, encounter sell signal

        #exit trade
        logs.append(-1)
        compare = 0
        exit_loss = 0

        #if trade was already exited before, check for disable trading flag here, do nothing here
        if disable_trading == 1:
            disable_trading = 0
            # compare = -1
            # print(f"disable_trading in buy trade - while exiting: {disable_trading}")
            logs[-1] = 0


    #SHORT TRADES
    elif df_train["flag"].iloc[i] == -1 and compare == 0:
        # No open trade, enounter sell siganl 
        exit_loss = 0
        sell_price = df_train["close"].iloc[i]
        
        compare = -1
        logs.append(-1)


    elif (df_train["flag"].iloc[i] != 1 and compare == -1):
        # Current sell trade, encounter sell signal or no signal - do nothing, update stop loss
        logs.append(0)

        #calculate pnl, if we exit now
        buy_price = df_train["close"].iloc[i]
        exit_loss = (sell_price - buy_price)/sell_price 
    

        #exit trade, if the loss is higher than stop loss
        if exit_loss < thresh and disable_trading == 0:
            logs[-1] = 1
            # print(f"disable_trading in sell trade - stop loss: {disable_trading}")
            # print(i)
            disable_trading = 1

    elif df_train["flag"].iloc[i] == 1 and compare == -1:
        # 
        # print("Current sell trade, encounter buy signal")
        # print(f"{disable_trading}")

        #exit trade
        logs.append(1)
        compare = 0
        exit_loss = 0

        #if trade was already exited before, check for disable trading flag here, do nothing here
        if disable_trading == 1:
            # print("Check")
            disable_trading = 0
            logs[-1] = 0
            # compare = 1

    elif df_train["flag"].iloc[i] == 0 and compare == 0:
        logs.append(0)

#close out positions (if needed)
logs[-1] = -np.sum(logs)

df_train["logs"] = np.array(logs)


In [126]:
df_trades = df_train[df_train["logs"] !=0 ]

sum(df_trades["logs"] == df_trades["flag"])

812

In [127]:
df_trades[:40]

,date_time,open,close,volume,Daily_Return,Next_2_Days_Return,RSI,exp_RSI,EMA_Slope,flag,logs
59,2018-01-01 09:30:00,13575.76,13601.01,55.300506,1.694653,0.477170,0.0,0.002934,-1.318575,-1.0,-1
143,2018-01-01 16:30:00,13199.95,13220.56,27.017572,1.632698,5.660880,0.0,0.002934,-1.356562,1.0,1
167,2018-01-01 18:30:00,12968.53,13017.00,25.255931,3.739057,-13.598371,0.0,0.002934,-0.989330,1.0,1
251,2018-01-02 01:30:00,13422.78,13399.24,21.525367,-1.703905,3.964404,0.0,0.002934,0.716906,-1.0,-1
299,2018-01-02 05:30:00,13750.01,13750.01,25.736793,0.000000,0.725090,0.0,0.002934,-0.131681,-1.0,-1
347,2018-01-02 09:30:00,13163.85,13127.31,52.891994,-4.742279,6.455245,0.0,0.002934,0.784236,1.0,1
455,2018-01-02 18:30:00,13847.49,13864.98,82.571610,1.263045,-4.038232,0.0,0.002934,-0.873736,-1.0,-1
695,2018-01-03 14:30:00,14597.93,14519.99,117.441555,-5.339113,-18.040646,0.0,0.002934,2.801950,1.0,1
707,2018-01-03 15:30:00,14465.96,14455.55,78.808246,-0.721002,5.876636,0.0,0.002934,-0.300171,1.0,1
743,2018-01-03 18:30:00,14880.00,14855.02,43.369038,-1.678763,0.724334,0.0,0.002934,0.117795,-1.0,-1


In [128]:

# rename column logs to signal
df_train.rename(columns={'logs': 'signal'}, inplace=True)


In [129]:
df_train.rename(columns={'date_time': 'datetime'}, inplace=True)

print(df_train[df_train['signal'] != 0])

df_train.to_csv("../src/logs/final.csv")

                  datetime      open     close       volume  Daily_Return  \
59     2018-01-01 09:30:00  13575.76  13601.01    55.300506      1.694653   
143    2018-01-01 16:30:00  13199.95  13220.56    27.017572      1.632698   
167    2018-01-01 18:30:00  12968.53  13017.00    25.255931      3.739057   
251    2018-01-02 01:30:00  13422.78  13399.24    21.525367     -1.703905   
299    2018-01-02 05:30:00  13750.01  13750.01    25.736793      0.000000   
...                    ...       ...       ...          ...           ...   
253201 2020-06-02 19:30:00   9506.06   9581.47  2835.031230      7.609545   
253213 2020-06-02 20:30:00   9477.90   9468.81   631.771547     -0.958019   
254701 2020-06-08 00:30:00   9746.61   9773.81  2110.096736      2.668294   
255781 2020-06-11 18:30:00   9639.65   9635.54   360.643502     -0.434659   
268164 2020-07-24 21:55:00   9538.18   9543.00    98.544875      0.504289   

        Next_2_Days_Return  RSI   exp_RSI  EMA_Slope  flag  signal  
59    